In [7]:
import psycopg2
import tkinter as tk
def connect():
    try:
        conn = psycopg2.connect(
            dbname="language_school",
            user="postgres",
            password="postgres",
            host="localhost"
        )
        print("Conexão ao banco de dados estabelecida com sucesso.")
        return conn
    except psycopg2.Error as e:
        print("Erro ao conectar:", e)
        return None

def close_connection(conn):
    if conn:
        conn.close()
        print("Conexão ao banco de dados fechada.")

def execute_query(conn, query):
    try:
        cur = conn.cursor()
        cur.execute(query)
        if query.strip().lower().startswith("select"):
            rows = cur.fetchall()
            for row in rows:
                print(row)
            return rows  # Retorna os resultados para consultas SELECT
        else:
            conn.commit()
            print("Operação executada com sucesso.")
            return None  # Retorna None para operações de INSERT
        cur.close()
    except psycopg2.Error as e:
        print("Erro ao executar a consulta:", e)


def delete_data_from_pessoa(conn, cpf):
    try:
        cur = conn.cursor()
        query = f"DELETE FROM pessoa WHERE CPF = '{cpf}'"
        cur.execute(query)
        conn.commit()
        cur.close()
        print("Deletado com sucesso")
    except psycopg2.Error as e:
        print(f"Erro ao deletar pessoa da tabela 'pessoa': {e}")

def insert_into_pessoa(connection, nome, cpf, rua, numero, cep, telefone, email):
    try:
        cur = connection.cursor()

        # Verifica se o CPF já existe na tabela pessoa
        cur.execute(f"SELECT COUNT(*) FROM pessoa WHERE CPF = '{cpf}'")
        count = cur.fetchone()[0]

        if count > 0:
            print("CPF já existe na base de dados. Não é possível inserir.")
            return None

        # Insere na tabela pessoa
        query = f"""
            INSERT INTO pessoa (Nome, CPF, Rua, Numero, CEP, Telefone, Email)
            VALUES ('{nome}', '{cpf}', '{rua}', '{numero}', '{cep}', '{telefone}', '{email}')
            RETURNING CPF
        """
        cur.execute(query)
        cpf_inserido = cur.fetchone()[0] if cur.rowcount > 0 else None

        cur.close()
        return cpf_inserido
    except psycopg2.Error as e:
        print("Erro ao inserir na tabela pessoa:", e)
        return None
# Conectar ao banco de dados
connection = connect()

if connection:
    while True:
        person_type, search_type, search_term = None, None, None  # Reinicializando as variáveis
        print("\nEscolha a operação:")
        print("1. Pesquisa Filtrada")
        print("2. Inserir dados")
        print("3. Atualizar dados")
        print("4. Deletar dados")
        print("5. Sair")

        choice = input("Digite o número da operação desejada: ")

        if choice == "1":
            
            # Perguntar qual tipo de pessoa procurar
            print("Escolha o tipo de pessoa:")
            print("1. Aluno")
            print("2. Professor")
            print("3. Funcionário")
            person_type = input("Digite o número correspondente: ")

            # Determinar a tabela correspondente ao tipo de pessoa
            if person_type == "1":
                table_name = "aluno"
                id_column = "id_aluno"
            elif person_type == "2":
                table_name = "professor"
                id_column = "id_professor"
            elif person_type == "3":
                table_name = "funcionario"
                id_column = "id_funcionario"
            else:
                print("Opção inválida.")
                continue

            # Perguntar que tipo de pesquisa fazer
            print("Escolha o tipo de pesquisa:")
            print("1. Por CPF")
            print("2. Por nome")
            print("3. Por ID")
            search_type = input("Digite o número correspondente: ")

            # Realizar a pesquisa de acordo com o tipo selecionado
            if search_type == "3":
                search_term = input(f"Digite o ID do {table_name}: ")
                query = f"""
                    SELECT p.Nome, p.CPF, {table_name}.{id_column}, p.Rua, p.Numero, p.CEP, p.Telefone, p.Email
                    FROM pessoa p 
                    JOIN {table_name} ON p.CPF = {table_name}.CPF 
                    WHERE {table_name}.{id_column} = '{search_term}'
                """
            elif search_type == "1":
                search_column = "CPF"
                search_term = input(f"Digite o CPF do {table_name}: ")
                query = f"""
                    SELECT p.Nome, p.CPF, p.Rua, p.Numero, p.CEP, p.Telefone, p.Email, {table_name}.{id_column}
                    FROM pessoa p 
                    LEFT JOIN {table_name} ON p.CPF = {table_name}.CPF 
                    WHERE p.{search_column} = '{search_term}'
                """
            elif search_type == "2":
                search_column = "Nome"
                search_term = input(f"Digite o Nome do {table_name}: ")
                query = f"""
                    SELECT p.Nome, p.CPF, p.Rua, p.Numero, p.CEP, p.Telefone, p.Email, {table_name}.{id_column}
                    FROM pessoa p 
                    LEFT JOIN {table_name} ON p.CPF = {table_name}.CPF 
                    WHERE p.{search_column} = '{search_term}'
                """
            else:
                print("Opção inválida")
                continue

            execute_query(connection, query)


        #inserir na tabela   
        elif choice == "2":
            print("Escolha o tipo de pessoa para inserir:")
            print("1. Aluno")
            print("2. Professor")
            print("3. Funcionário")
            person_type = input("Digite o número correspondente: ")

            # Coleta de dados pessoais
            nome = input("Digite o nome da pessoa: ")
            cpf = input("Digite o CPF da pessoa: ")
            rua = input("Digite a rua: ")
            numero = input("Digite o número: ")
            cep = input("Digite o CEP: ")
            telefone = input("Digite o telefone: ")
            email = input("Digite o email: ")

            # Chama a função para inserir na tabela pessoa
            cpf_inserido = insert_into_pessoa(connection, nome, cpf, rua, numero, cep, telefone, email)

            # Verifica se houve sucesso na inserção na pessoa antes de seguir
            if cpf_inserido:
                # Inserção na tabela específica associada
                if person_type == "1":
                    query_aluno = f"INSERT INTO aluno (CPF) VALUES ('{cpf_inserido}')"
                    execute_query(connection, query_aluno)
                elif person_type == "2":
                    qualificacao = input("Digite a qualificação: ")
                    query_professor = f"INSERT INTO professor (CPF, qualificacao) VALUES ('{cpf_inserido}', '{qualificacao}')"
                    execute_query(connection, query_professor)
                elif person_type == "3":
                    cargo = input("Digite o cargo: ")
                    query_funcionario = f"INSERT INTO funcionario (CPF, cargo) VALUES ('{cpf_inserido}', '{cargo}')"
                    execute_query(connection, query_funcionario)
                else:
                    print("Opção inválida.")
                    continue
            else:
                print("Erro ao inserir na tabela pessoa. Operação abortada.")
        #Atualizar dados da tabela    
        elif choice == "3":
                print("Escolha o tipo de pessoa para atualizar:")
                print("1. Aluno")
                print("2. Professor")
                print("3. Funcionário")
                person_type = input("Digite o número correspondente: ")

                if person_type == "1":
                    table_name = "aluno"
                    id_column = "id_aluno"
                elif person_type == "2":
                    table_name = "professor"
                    id_column = "id_professor"
                elif person_type == "3":
                    table_name = "funcionario"
                    id_column = "id_funcionario"
                else:
                    print("Opção inválida.")
                    continue

                id_to_update = input(f"Digite o ID do(a) {table_name}: ")
                new_name = input("Digite o novo Nome: ")
                new_rua = input("Digite a nova Rua: ")
                new_numero = input("Digite o novo numero da residencia: ")
                new_cep   =  input("Digite o novo CEP: ")
                new_telefone = input("Digite o novo telefone: ")
                new_email    = input("Digite o novo email")
            

                # Atualizar os dados pessoais na tabela pessoa
                query = f"""
                    UPDATE pessoa
                    SET Nome = '{new_name}',  Rua = '{new_rua}',
                        Numero ='{new_numero}',CEP ='{new_cep}',Telefone ={new_telefone},
                        Email = '{new_email}'
                
                    WHERE CPF IN (
                        SELECT CPF FROM {table_name} WHERE {id_column} = {id_to_update}
                    )
                """
                execute_query(connection, query)

        if choice == "4":
            print("Escolha a tabela para deletar os dados:")
            print("1. Aluno")
            print("2. Professor")
            print("3. Funcionário")
            table_choice = input("Digite o número correspondente: ")

            if table_choice == "1":
                table_name = "aluno"
                id_column = "id_aluno"
            elif table_choice == "2":
                table_name = "professor"
                id_column = "id_professor"
            elif table_choice == "3":
                table_name = "funcionario"
                id_column = "id_funcionario"
            else:
                print("Opção inválida.")
                continue

            search_term = input(f"Digite o {id_column} do registro a ser deletado: ")

            # Obtenha o CPF do registro a ser deletado na tabela 'aluno', 'professor' ou 'funcionario'
            cpf = execute_query(connection, f"SELECT CPF FROM {table_name} WHERE {id_column} = '{search_term}'")

            if cpf:
                cpf = cpf[0][0]
                delete_data_from_pessoa(connection, cpf)
            else:
                print("Registro não encontrado.")

        elif choice == "5":
            print("Encerrando o programa.")
            close_connection(connection)
            break
        else:
            print("Opção inválida. Por favor, escolha uma operação válida.")



Conexão ao banco de dados estabelecida com sucesso.

Escolha a operação:
1. Pesquisa Filtrada
2. Inserir dados
3. Atualizar dados
4. Deletar dados
5. Sair


Erro ao executar a consulta: relation "sua_tabela" does not exist
LINE 1: SELECT * FROM sua_tabela WHERE ID = '1'
                      ^

Opção inválida. Por favor, escolha uma operação válida.

Escolha a operação:
1. Pesquisa Filtrada
2. Inserir dados
3. Atualizar dados
4. Deletar dados
5. Sair
Encerrando o programa.
Conexão ao banco de dados fechada.


In [8]:
import tkinter as tk
import psycopg2

# Função para executar a consulta com base nos filtros inseridos
def execute_search():
    # Obtenha os valores dos campos de entrada
    num_matricula = entry_num_matricula.get()
    
    # Montar a consulta com base nos valores inseridos nos campos
    query = f"""
        SELECT *
        FROM view_frequencia_aluno
        WHERE num_matricula = '{num_matricula}'
    """
    
    # Executar a consulta
    results = execute_database_query(connection, query)
    
    # Exibir os resultados na janela
    if results:
        for row in results:
            # Exibir os resultados formatados
            print(f"Número de Matrícula: {row[0]}")
            print(f"Nome do Aluno: {row[1]}")
            print(f"Quantidade de Presenças: {row[2]}")
            print(f"Quantidade de Ausências: {row[3]}")
            print("\n")  # Adicione uma linha em branco para separar os resultados
    else:
        print("Nenhum resultado encontrado.")

# Função para executar a consulta no banco de dados
def execute_database_query(conn, query):
    try:
        cur = conn.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        cur.close()
        return rows
    except psycopg2.Error as e:
        print("Erro ao executar a consulta:", e)
        return None

# Função para conectar ao banco de dados
def connect():
    try:
        conn = psycopg2.connect(
            dbname="language_school",
            user="postgres",
            password="postgres",
            host="localhost"
        )
        print("Conexão ao banco de dados estabelecida com sucesso.")
        return conn
    except psycopg2.Error as e:
        print("Erro ao conectar:", e)
        return None

# Conectar ao banco de dados
connection = connect()

# Criar a janela
root = tk.Tk()
root.title("Acessar View")

# Campos de filtro
label_num_matricula = tk.Label(root, text="Número da Matrícula:")
label_num_matricula.pack()
entry_num_matricula = tk.Entry(root)
entry_num_matricula.pack()

# Botão para executar a consulta
btn_execute = tk.Button(root, text="Executar Consulta", command=execute_search)
btn_execute.pack()

# Iniciar a interface
root.mainloop()


Conexão ao banco de dados estabelecida com sucesso.
Nenhum resultado encontrado.
Número de Matrícula: 12
Nome do Aluno: Exemplo Aluno
Quantidade de Presenças: 0
Quantidade de Ausências: 0


Nenhum resultado encontrado.
Número de Matrícula: 10
Nome do Aluno: Tatiana Soares
Quantidade de Presenças: 44
Quantidade de Ausências: 0


Número de Matrícula: 6
Nome do Aluno: Mariana Oliveira
Quantidade de Presenças: 42
Quantidade de Ausências: 0


Número de Matrícula: 4
Nome do Aluno: Ana Alves
Quantidade de Presenças: 41
Quantidade de Ausências: 1


Número de Matrícula: 2
Nome do Aluno: Maria Santos
Quantidade de Presenças: 38
Quantidade de Ausências: 4


